In [1]:
import os
import numpy as np
from sippy import functionset as fset
import torch.nn.functional as F
import torch

from simba.model import Simba
from simba.functions import generate_random_system, generate_data, add_noise, identify_baselines, matlab_baselines
from simba.util import print_all_perf, eval_simba, fix_seed, save_results

from simba.parameters import base_parameters, baselines_to_use
parameters = base_parameters

In [2]:
# To modify
seed = 1
parameters['max_epochs'] = 10000
parameters['init_epochs'] = 150000
parameters['device'] = 'cpu'
parameters['init_from_matlab_or_ls'] = True
directory = os.path.join('saves', f'Random_init_{seed}')
parameters['id_D'] = True
parameters['print_each'] = 1000


In [3]:
fix_seed(seed)

# Data parameters
number_trajectories = 1
number_trajectories_test = 1
nx = 5
nu = 3
ny = 3

N = 300
dt = 0.05
min_eigenvalue = 0.5

process_noise = False
colored = False
process_noise_scale = 0.1
output_noise_scale = 0.25

# Simba parameters for this simulation
parameters['input_output'] = True
parameters['stable_A'] = True
parameters['LMI_A'] = True

parameters['grad_clip'] = 100
parameters['learning_rate'] = 0.001
parameters['dropout'] = 0.2
parameters['batch_size'] = 128 

parameters['learn_x0'] = False

# The system is a random DISCRETE system, so we set delta = None since A is not
# expected to be identified from a continuous system, i.e., with form I + delta*A
parameters['delta'] = None

path_to_matlab = parameters['path_to_matlab']

In [4]:
# Prepare the data
for j in range(10):
    A, B, C, D = generate_random_system(nx=nx, nu=nu, ny=ny, N=N, stable_A=parameters['stable_A'], min_eigenvalue=min_eigenvalue)

    x0 = np.zeros((number_trajectories,1,nx))
    x0_val = x0
    x0_test = np.zeros((number_trajectories_test,1,nx))
    U = np.zeros((number_trajectories,N,nu))
    U_val = np.zeros((number_trajectories,N,nu))
    U_test = np.zeros((number_trajectories_test,N,nu))
    X = np.zeros((number_trajectories,N,nx))
    X_val = np.zeros((number_trajectories,N,nx))
    X_test = np.zeros((number_trajectories_test,N,nx))
    Y = np.zeros((number_trajectories,N,ny))
    Y_val = np.zeros((number_trajectories,N,ny))
    Y_test = np.zeros((number_trajectories_test,N,ny))

    for t in range(number_trajectories):
        # Creating exciting input sequences
        for i in range(nu):
            U[t, :,i],_,_ =  fset.GBN_seq(N, 0.1)
            U_val[t, :,i],_,_ =  fset.GBN_seq(N, 0.1)
        # Simulate the system to create the data
        U[t,:,:], Y[t,:,:], X[t,:,:] = generate_data(A, B, C, D, N, parameters['id_D'], U=U[t,:,:], x0=x0[t,:,:], gaussian_U=False, process_noise_scale=process_noise_scale, dt=dt)
        U_val[t,:,:], Y_val[t,:,:], X_val[t,:,:] = generate_data(A, B, C, D, N, parameters['id_D'], U=U_val[t,:,:], x0=x0_val[t,:,:], gaussian_U=False, process_noise_scale=process_noise_scale, dt=dt)

    for t in range(number_trajectories_test):
        # Creating exciting input sequences
        for i in range(nu):
            U_test[t, :,i],_,_ =  fset.GBN_seq(N, 0.1)
        # Simulate the system to create the data
        U_test[t,:,:], Y_test[t,:,:], X_test[t,:,:] = generate_data(A, B, C, D, N, parameters['id_D'], U=U_test[t,:,:], x0=x0_test[t,:,:], gaussian_U=False, process_noise_scale=process_noise_scale, dt=dt)

    # Add output noise if wanted
    if output_noise_scale > 0:
        Y = add_noise(Y, voss=False, colored=colored, scale=output_noise_scale)

    # Store all the parameters for reproducibility
    sim_params = (seed, process_noise_scale, output_noise_scale, number_trajectories, number_trajectories_test)
    data_params = (A, B, C, D, N, parameters['id_D'])
    data = (U, U_val, U_test, X, X_val, X_test, Y, Y_val, Y_test, x0, x0_val, x0_test)
    
    # Baselines
    x0 = x0_val = x0_test = np.zeros((1,1,nx))
    names, baselines, times, train_ids, validation_ids, test_ids = identify_baselines(nx=nx, U=U, U_val=U_val, U_test=U_test, Y=Y, Y_val=Y_val, Y_test=Y_test,
                                                                            x0=x0, x0_val=x0_val, x0_test=x0_test, dt=dt,
                                                                            parameters=parameters, baselines_to_use=baselines_to_use)

    # Simba
    name = f'SIMBa_{j}'
    simba = Simba(nx=nx, nu=nu, ny=ny, parameters=parameters)
    simba.fit(U, U_val=U_val, U_test=U_test, X=X, X_val=X_val, X_test=X_test, Y=Y, Y_val=Y_val, Y_test=Y_test, x0=x0, x0_val=x0_val, x0_test=x0_test, baselines_to_use=baselines_to_use)
    simba.save(directory=directory, save_name=name)

    names, times, train_ids, validation_ids, test_ids = eval_simba(simba, name, names, times, train_ids, validation_ids, test_ids,
                                                        U, U_val, U_test, X, X_val, X_test, Y, Y_val, Y_test, x0, x0_val, x0_test)
    save_results(directory=directory, save_name=f'Results_{j}', names=names, times=times, train_ids=train_ids, 
                validation_ids=validation_ids, test_ids=test_ids, data=data)
    
    print_all_perf(names, times, train_ids, validation_ids, test_ids, Y, Y_val, Y_test)



Initilization starts, fitting A!
Epoch	Fitting loss
1	3.29E-01
50000	1.62E-06
100000	3.61E-07
150000	2.52E-07
Total initialization time:	01'16"
Best loss at epoch 127022:	7.35E-08

SIPPY-PARSIM-K performance (Train and validation are only measured on the
first trajectory if there are several for now):
Train loss	Val loss	Test loss
6.78E-01	6.59E-01	6.72E-01

Training of SIMBa starts!
Training data shape:	(1, 300, *)
Validation data shape:	(1, 300, *)
Test data shape:	(1, 300, *)

Epoch	Train loss	Val loss	Test loss
1	5.44E-01	7.00E-01	6.90E-01
1000	4.72E-01	6.65E-01	6.82E-01
2000	5.23E-01	6.76E-01	6.80E-01
3000	5.01E-01	6.65E-01	6.74E-01
4000	5.32E-01	6.72E-01	6.86E-01
5000	5.23E-01	6.75E-01	6.81E-01
6000	5.32E-01	6.81E-01	6.95E-01
7000	5.20E-01	6.80E-01	6.87E-01
8000	5.48E-01	6.70E-01	6.89E-01
9000	5.25E-01	6.76E-01	6.87E-01
10000	5.47E-01	6.68E-01	6.74E-01

Average time per 100 epochs:	08"
Total training time:		14'52"

Best model performance:
2240	5.28E-01	6.52E-01	6.79E-01

Method	